## Exemplo de segmentação utilizando Detectron2

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

O [dataset](https://universe.roboflow.com/nutritionverse/mealsynth) utilizado neste exemplo é constituído por 913 imagens (49 classes) de diferentes alimentos.

Treino: 624

Validação: 265

Teste: 24



## Aceder ao Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive/') # nome da pasta onde serão colocados os ficheiros do Google Drive -> /nome_da_pasta/MyDrive/

## Instalar Detectron2



In [ ]:
# instalar repositório através do Git
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Bibliotecas

In [ ]:
# Geral
import os
import cv2

from datetime import datetime
from google.colab.patches import cv2_imshow

# Preparação do dataset
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# Visualização
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

# Configuração
from detectron2 import model_zoo
from detectron2.config import get_cfg

# Avaliação
from detectron2.engine import DefaultPredictor

# Treino
from detectron2.engine import DefaultTrainer

# Dataset

Fazer unzip do dataset

In [ ]:
# Unzip do dataset
!unzip /content/drive/MyDrive/Exemplo_Codigo/datasets/comida.zip -d /content/data/

# Localização do dataset

In [ ]:
DATA_SET_NAME = "comida" # nome do dataset
ANNOTATIONS_FILE_NAME = "_annotations.coco.json" # anotações
data_location = "/content/data/comida/" # caminho do dataset

In [ ]:
# Dados de treino
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(data_location, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(data_location, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TRAIN_DATA_SET_NAME,
    metadata={},
    json_file=TRAIN_DATA_SET_ANN_FILE_PATH,
    image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
)

# Dados de teste
TEST_DATA_SET_NAME = f"{DATA_SET_NAME}-test"
TEST_DATA_SET_IMAGES_DIR_PATH = os.path.join(data_location, "test")
TEST_DATA_SET_ANN_FILE_PATH = os.path.join(data_location, "test", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TEST_DATA_SET_NAME,
    metadata={},
    json_file=TEST_DATA_SET_ANN_FILE_PATH,
    image_root=TEST_DATA_SET_IMAGES_DIR_PATH
)

# Dados de validação
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(data_location, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(data_location, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name = VALID_DATA_SET_NAME,
    metadata = {},
    json_file = VALID_DATA_SET_ANN_FILE_PATH,
    image_root = VALID_DATA_SET_IMAGES_DIR_PATH
)

# Visualizar dados de treino

In [ ]:
# Exemplo de dados de treino

metadata = MetadataCatalog.get(TRAIN_DATA_SET_NAME)
dataset_train = DatasetCatalog.get(TRAIN_DATA_SET_NAME)

dataset_entry = dataset_train[0]
image = cv2.imread(dataset_entry["file_name"])

visualizer = Visualizer(
    image[:, :, : : -1],
    metadata = metadata,
    scale = 0.8,
    instance_mode = ColorMode.IMAGE_BW
)

out = visualizer.draw_dataset_dict(dataset_entry)
cv2_imshow(out.get_image()[:, :, : : -1])

# Treinar modelo


## Configuração

In [ ]:
# Parâmetros
ARCHITECTURE = "mask_rcnn_R_101_FPN_3x" # arquitetura utilizada no treino
CONFIG_FILE_PATH = f"COCO-InstanceSegmentation/{ARCHITECTURE}.yaml" # configuração do modelo
MAX_ITER = 1000 # número de iterações
EVAL_PERIOD = 200 # patience
BASE_LR = 0.001 # learning rate
NUM_CLASSES = 49 # número de classes

# Output
OUTPUT_DIR_PATH = os.path.join(
    DATA_SET_NAME,
    ARCHITECTURE,
    datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
)

os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH) # pesos
cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,) # dados de treino
cfg.DATASETS.TEST = (TEST_DATA_SET_NAME,) # dados de teste
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32 # feature maps por cada imagem
cfg.TEST.EVAL_PERIOD = EVAL_PERIOD # gravar os resultados a cada x iterações
cfg.DATALOADER.NUM_WORKERS = 2 # número de workers a trabalhar -> carregar os dados
cfg.SOLVER.IMS_PER_BATCH = 2 # batch utilizada em cada iteração
cfg.INPUT.MASK_FORMAT='bitmask' # apenas a máscara se encontra a cores
cfg.SOLVER.BASE_LR = BASE_LR
cfg.SOLVER.MAX_ITER = MAX_ITER
cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
cfg.OUTPUT_DIR = OUTPUT_DIR_PATH # caminho dos outputs

## Treino

In [ ]:
trainer = DefaultTrainer(cfg) # modelo de treino
trainer.resume_or_load(resume=False)
trainer.train() # treino

## Avaliação

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") # pesos
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # confiança mínima de cada feature map
predictor = DefaultPredictor(cfg) # modelo de previsões

In [ ]:
# inferências

# utilizar imagens de teste
img = cv2.imread("")
outputs = predictor(img)

visualizer = Visualizer(
    img[:, :, ::-1],
    metadata=metadata,
    scale=0.8,
    instance_mode=ColorMode.IMAGE_BW
)
out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

